# SDXLIB Demonstration

**The intention of this notebook is to demonstrate the most basic methods available in the SDXLIB library on the most basic instance accepted by the API. Future notebooks will demonstrate the optional attributes and additional error handling.**

Once the API is in place, requests_mock and config should no longer be necessary.

In [1]:
import json
from pprint import pprint
import requests
import requests_mock

from config import *
from sdxlib.sdx_client import SDXClient
from sdxlib.sdx_exception import SDXException
from sdxlib.sdx_response import SDXTopologyResponse
from sdxlib.sdx_topology import *

# Creating L2VPN

### Demonstration of L2VPN Client Creation and Error Handling

**This portion of the demo will create an 'SDXClient' instance using the minimum required attributes for the API request:**
* URL - "http://example.com"
* Name - Any string that is less than 50 characters.
* Endpoints - A list of dictionaries. Each dictionary must contain the following keys:
    * port_id - The Uniform Resource Name (URN) of a network device's port. Must follow the pattern: "urn:sdx:port:<oxp_url>:<node_name>:<port_name>"
    * vlan: Describes how the SDX and OXPs should treat L2VPN frames. Accepted values are:
        * "any"
        * "all"
        * "untagged"
        * integer string
        * VLAN range in the format "VLAN ID1:VLAN ID2"

In [2]:
# Define the required attributes
url = "http://aw-sdx-controller.renci.org:8081"
client_name = "Test L2VPN"
client_endpoints = [
    {"port_id": "urn:sdx:port:sax.br:Rtr01:50", "vlan": "any"},
    {"port_id": "urn:sdx:port:ampath.net:Ampath3:50", "vlan": "any"}
]

In [3]:
# Create the L2VPN object
client = SDXClient(url, client_name, client_endpoints)

In [4]:
client

SDXClient(name=Test L2VPN, endpoints=[{'port_id': 'urn:sdx:port:sax.br:Rtr01:50', 'vlan': 'any'}, {'port_id': 'urn:sdx:port:ampath.net:Ampath3:50', 'vlan': 'any'}], description=None, notifications=None, scheduling=None, qos_metrics=None, base_url=http://aw-sdx-controller.renci.org:8081)

### Error Handling

**The below mocks the call to the API which should return an error code of 409, indicating that the specified service already exists.**

In [5]:
# Create a mocker for requests for the unsuccessful case
with requests_mock.Mocker() as mock_requests:
    # Mock unsuccessful L2VPN creation
    mock_requests.register_uri(
        "POST", f"{url}/l2vpn/1.0",
        json=mock_response_unsuccessful_409,
        status_code=409  # Set status code for unsuccessful creation (Options are: 400, 401, 402, 409, 410, 411, 422)
    )
    
    # Debug print to verify the mock setup
    print(f"Request URL: {url}/l2vpn/1.0")

    try:
        response = client.create_l2vpn()  
        print("Request successful.")
    except SDXException as e:
        print(f"L2VPN creation failed: {e}")  

Failed to create L2VPN. Status code: 409: L2VPN Service already exists


Request URL: http://aw-sdx-controller.renci.org:8081/l2vpn/1.0
L2VPN creation failed: SDXException: L2VPN Service already exists (status_code=409)


**This example will demonstrate a 'Connection error'. This should cause an SDXException.**

In [6]:
# Create a mocker for requests to simulate a RequestException
with requests_mock.Mocker() as mock_requests:
    # Mock a request exception scenario (e.g., connection error)
    mock_requests.register_uri(
        "POST", f"{url}/l2vpn/1.0",
        exc=requests.exceptions.RequestException("Connection error")
    )
    
    # Debug print to verify the mock setup
    print(f"Request URL: {url}/l2vpn/1.0")

    try:
        response = client.create_l2vpn()  
    except SDXException as e:
        print(f"L2VPN creation failed: {e}")  
    except requests.exceptions.RequestException as e:
        print(f"Request exception occurred: {e}")  

An error occurred while creating L2VPN: Connection error


Request URL: http://aw-sdx-controller.renci.org:8081/l2vpn/1.0
L2VPN creation failed: SDXException: An error occurred while creating L2VPN: Connection error (status_code=None)


### Successful Call to the API

**This call should return a response from the API with the service_id attribute which has a value in the form of UUID.**

In [7]:
# Create a mocker for requests
with requests_mock.Mocker() as mock_requests:
    # Mock successful L2VPN creation
    mock_requests.register_uri(
        "POST", f"{url}/l2vpn/1.0", 
        json=mock_response_successful,
        status_code=201  # Set status code for successful creation
    )
    
    # Debug statement to confirm URL and method
    print(f"Request URL: {url}/l2vpn/1.0")

    try:
        response = client.create_l2vpn()  
        print("L2VPN creation successful!")
        pprint(response)
    except SDXException as e:
        print(f"L2VPN creation failed: {e}")  

Request URL: http://aw-sdx-controller.renci.org:8081/l2vpn/1.0
L2VPN creation successful!
{'service_id': '123e4567-e89b-12d3-a456-426614174000'}


# Update L2VPNs

### Demonstration of L2VPN Client Update and Error Handling

**Again, create a client instance with the minimum required attributes for a successful API call.**

The update method requires the service_id and any attributes that are to be updated. The attributes that may be updated are: name, endpoints, description, notifications, scheduling, qos_metrics, and the state attribute that is returned from the API. The state attribute may be updated to "enabled" or "disabled".

In [8]:
# Define your SDXClient instance with mock URL and initial configuration
url = "http://aw-sdx-controller.renci.org:8081"

In [9]:
# Create the SDXClient instance
client = SDXClient(url)

### Error Handling

**Here, we are going to attempt an update to the name, description, and state attributes. This example should demonstrate the returned exception for a 404 response, the supplied service_id is not found.**

In [10]:
# Example update_l2vpn scenario with requests_mock
with requests_mock.Mocker() as m:

    # Mock endpoint URL
    mock_url = f"{url}/l2vpn/{client.VERSION}/123e4567-e89b-12d3-a456-426614174000"
    
    # Example for handling other status codes if needed
    m.patch(mock_url, status_code=404)  # Mocking 404 response
    
    print(f"Request URL: {mock_url}")
    
    try:
        response = client.update_l2vpn(service_id="123e4567-e89b-12d3-a456-426614174000", name="New L2VPN Name", description="Updated description", state="enabled")
        print("Update Successful:")
        pprint(response)
    except SDXException as e:
        print("SDXException:", e)
    except Exception as e:
        print("Exception:", e)

Failed to update L2VPN. Status code: 404: L2VPN Service ID not found


Request URL: http://aw-sdx-controller.renci.org:8081/l2vpn/1.0/123e4567-e89b-12d3-a456-426614174000
SDXException: SDXException: L2VPN Service ID not found (status_code=404)


**And here I expect to see a successful response, with a description indicating that the L2VPN has been modified along with the service_id for the modified object.**

In [11]:
# Example update_l2vpn scenario with requests_mock
with requests_mock.Mocker() as m:

    # Mock endpoint URL
    mock_url = f"{url}/l2vpn/{client.VERSION}/123e4567-e89b-12d3-a456-426614174000"
    
    # Register mock response for patch request
    m.patch(mock_url, json=mock_response, status_code=200)
    
    print(f"Request URL: {mock_url}")
    
    # Call the update_l2vpn method
    try:
        response = client.update_l2vpn(service_id="123e4567-e89b-12d3-a456-426614174000", name="New L2VPN Name", description="Updated description", state="enabled")
        print("Update Successful")
        pprint(response)
    except SDXException as e:
        print("SDXException:", e)
    except Exception as e:
        print("Exception:", e)


Request URL: http://aw-sdx-controller.renci.org:8081/l2vpn/1.0/123e4567-e89b-12d3-a456-426614174000
Update Successful
None


# Retrieve All L2VPNs

### Demonstration to retrieve all L2VPN Clients and Error Handling

**Now we will query the API for a list of all available L2VPNs. Therefore, we will create a client that is empty except for the URL.**

There are two retrieval methods, one to retreive a list of all L2VPNs and one to retrieve the information for a single L2VPN (we will discuss this one next). 

In [12]:
client = SDXClient(url)

### Error Handling

**This error should simulate a network error.** 

In [13]:
with requests_mock.Mocker() as mock_requests:
    # Simulate a RequestException (e.g., network error)
    mock_requests.register_uri(
        "GET", f"{url}/l2vpn/{client.VERSION}",
        exc=requests.RequestException("Simulated network error")
    )
    
    print(f"Request URL: {url}/l2vpn/{client.VERSION}")

    # Call the get_all_l2vpns method
    try:
        response = client.get_all_l2vpns() 
        print("Get All L2VPNs (RequestException):")
        pprint(response)
    except SDXException as e:
        print(e)

Failed to retrieve L2VPN(s): Simulated network error


Request URL: http://aw-sdx-controller.renci.org:8081/l2vpn/1.0
SDXException: Unknown error occurred. (status_code=Failed to retrieve L2VPN(s): Simulated network error)


**Here I expect to see an empty dictionary because there are no active layer 2 VPNs.**

In [14]:
# Mock the get_all_l2vpns method
with requests_mock.Mocker() as mock_requests:
    # Mock response for no L2VPNs existing
    mock_requests.register_uri(
        "GET", f"{url}/l2vpn/{client.VERSION}",
        json=mock_response_no_l2vpns,
        status_code=200
    )
    
    print(f"Request URL: {url}/l2vpn/{client.VERSION}")
    
    # Call the get_all_l2vpns method
    try:
        response = client.get_all_l2vpns()
        print("Get All L2VPNs:", response)
    except SDXException as e:
        print("SDXException:", e)

Request URL: http://aw-sdx-controller.renci.org:8081/l2vpn/1.0
Get All L2VPNs: Empty DataFrame
Columns: []
Index: []


**Here I expect to see only the active l2vpn.**

In [15]:
with requests_mock.Mocker() as mock_requests:
    # Mock response for one or more archived L2VPNs existing
    mock_requests.register_uri(
        "GET", f"{url}/l2vpn/{client.VERSION}",
        json=mock_response_active_l2vpns_exist,
        status_code=200
    )
    
    print(f"Request URL: {url}/l2vpn/{client.VERSION}")
    
    # Call the get_all_l2vpns method
    try:
        response = client.get_all_l2vpns()
        print("Get All L2VPNs (One or more L2VPNs exist):")
        # pprint(response)
    except SDXException as e:
        print("SDXException:", e)

Request URL: http://aw-sdx-controller.renci.org:8081/l2vpn/1.0
Get All L2VPNs (One or more L2VPNs exist):


In [16]:
response

,service_id,name,endpoints,ownership,creation_date,archived_date,status,state,counters_location,last_modified,current_path,oxp_service_ids,description,notifications,scheduling,qos_metrics
0,c73da8e1-5d03-4620-a1db-7cdf23e8978c,VLAN between AMPATH/300 and TENET/150,"[{'port_id': 'urn:sdx:port:tenet.ac.za:Tenet03:50', 'vlan': '150'}, {'port_id': 'urn:sdx:port:sax.net:Sax02:50', 'vlan': '400'}, {'port_id': 'urn:sdx:port:ampath.net:Ampath3:50', 'vlan': '300'}]",user1,20240522T00:00:00Z,0,up,enabled,https://my.aw-sdx.net/l2vpn/7cdf23e8978c,0,[urn:sdx:link:tenet.ac.za:LinkToAmpath],"{'ampath.net': ['c73da8e1'], 'Tenet.ac.za': ['5d034620']}",Example 1,"[{'email': 'user@domain.com'}, {'email': 'user2@domain2.com'}]",None,"{'min_bw': {'value': 5, 'strict': False}, 'max_delay': {'value': 150, 'strict': True}}"
1,fa2c99ca-30a9-4b51-8491-683c52e326a6,Example 2,"[{'port_id': 'urn:sdx:port:tenet.ac.za:Tenet03:50', 'vlan': '3500'}, {'port_id': 'urn:sdx:port:sax.net:Sax01:50', 'vlan': '3500'}, {'port_id': 'urn:sdx:port:ampath.net:Ampath3:50', 'vlan': '3500'}]",user2,20240422T00:00:00Z,0,up,disabled,https://my.aw-sdx.net/l2vpn/52e326a6,0,"[urn:sdx:link:tenet.ac.za:LinkToSAX, urn:sdx:link:tenet.ac.za:LinkToAmpath, urn:sdx:link:ampath.net:LinkToSAX]","{'ampath.net': ['d82da7f9'], 'tenet.ac.za': ['ab034673'], 'sax.br': ['bb834633']}",None,None,None,None


**Here I expect to see only the archived l2vpns.**

In [17]:
with requests_mock.Mocker() as mock_requests:
    # Mock response for one or more archived L2VPNs existing
    mock_requests.register_uri(
        "GET", f"{url}/l2vpn/{client.VERSION}/archived",
        json=mock_response_archived_l2vpns_exist,
        status_code=200
    )
    
    print(f"Request URL: {url}/l2vpn/{client.VERSION}/archived")
    
    # Call the get_all_l2vpns method
    try:
        response = client.get_all_l2vpns(archived=True)
        print("Get All L2VPNs (One or more L2VPNs exist):")
        # pprint(response)
    except SDXException as e:
        print("SDXException:", e)

Request URL: http://aw-sdx-controller.renci.org:8081/l2vpn/1.0/archived
Get All L2VPNs (One or more L2VPNs exist):


# Retrieve a Specific L2VPN

### Demonstration to retrieve a specific L2VPN Client

**Now we will query the API for a specific L2VPN. We will create a client that is empty except for the URL.**

In [18]:
client = SDXClient(url)

**I will query for the l2vpn using the service_id attribute. With this request, I should see the complete response object from the API for this l2vpn.**

In [19]:
service_id = "c73da8e1-5d03-4620-a1db-7cdf23e8978c"

with requests_mock.Mocker() as mock_requests:
    # Simulate a RequestException (e.g., network error)
    mock_requests.register_uri(
        "GET", f"{url}/l2vpn/{client.VERSION}/{service_id}",
        json=mock_response_l2vpn_exists,
        status_code=200
    )
    
    print(f"Request URL: {url}/l2vpn/{client.VERSION}/{service_id}")

    # Call the get_l2vpn method
    try:
        response = client.get_l2vpn(service_id)
        print("Get L2VPN (Successful):")
    except SDXException as e:
        print("SDXException:", e)

response

Request URL: http://aw-sdx-controller.renci.org:8081/l2vpn/1.0/c73da8e1-5d03-4620-a1db-7cdf23e8978c
Get L2VPN (Successful):


service_id,name,port_id,vlan,description,qos_metrics,notifications,ownership,creation_date,archived_date,status,state,counters_location,last_modified,current_path,oxp_service_ids
c73da8e1-5d03-4620-a1db-7cdf23e8978c,VLAN between AMPATH/300 and TENET/150,urn:sdx:port:tenet.ac.za:Tenet03:50,150,Example 1,"{'min_bw': {'value': 5, 'strict': False}, 'max_delay': {'value': 150, 'strict': True}}","[{'email': 'user@domain.com'}, {'email': 'user2@domain2.com'}]",user1,20240522T00:00:00Z,0,up,enabled,https://my.aw-sdx.net/l2vpn/7cdf23e8978c,0,['urn:sdx:link:tenet.ac.za:LinkToAmpath'],"{'ampath.net': ['c73da8e1'], 'Tenet.ac.za': ['5d034620']}"
c73da8e1-5d03-4620-a1db-7cdf23e8978c,VLAN between AMPATH/300 and TENET/150,urn:sdx:port:ampath.net:Ampath3:50,300,Example 1,"{'min_bw': {'value': 5, 'strict': False}, 'max_delay': {'value': 150, 'strict': True}}","[{'email': 'user@domain.com'}, {'email': 'user2@domain2.com'}]",user1,20240522T00:00:00Z,0,up,enabled,https://my.aw-sdx.net/l2vpn/7cdf23e8978c,0,['urn:sdx:link:tenet.ac.za:LinkToAmpath'],"{'ampath.net': ['c73da8e1'], 'Tenet.ac.za': ['5d034620']}"


# Delete an L2VPN

### Demonstration to delete an L2VPN Client and Error Handling

**We will delete an l2vpn client using the service_id attribute.**

We will begin by creating a client instance that is empty except for the URL. 

In [20]:
client = SDXClient(url)

**This will simulate a successful deletion. The return on success is None.**

In [21]:
service_id = "c73da8e1-5d03-4620-a1db-7cdf23e8978c"

with requests_mock.Mocker() as mock_requests:
    # Mock response for successful deletion
    mock_requests.delete(
        f"{url}/l2vpn/{client.VERSION}/{service_id}",
        status_code=201
    )
    
    print(f"Request URL: {url}/l2vpn/{client.VERSION}/{service_id}")

    try:
        response = client.delete_l2vpn(service_id)
        print("Delete L2VPN (Successful):")
        pprint(response)
    except SDXException as e:
        print("SDXException:", e)

Request URL: http://aw-sdx-controller.renci.org:8081/l2vpn/1.0/c73da8e1-5d03-4620-a1db-7cdf23e8978c
Delete L2VPN (Successful):
None


### Error Handling

**This will demonstrate the return if the service_id provided for deletion does not exist.**

In [22]:
with requests_mock.Mocker() as mock_requests:
    # Mock response for 404: Service ID does not exist
    mock_requests.delete(
        f"{url}/l2vpn/{client.VERSION}/{service_id}",
        status_code=404,
        json={"description": "L2VPN Service ID provided does not exist"}
    )
    
    print(f"Request URL: {url}/l2vpn/{client.VERSION}/{service_id}")

    try:
        response = client.delete_l2vpn(service_id)
        print("Delete L2VPN (Service ID not found):")
        pprint(response)
    except SDXException as e:
        print("SDXException:", e)

Failed to delete L2VPN. Status code: 404: L2VPN Service ID provided does not exist


Request URL: http://aw-sdx-controller.renci.org:8081/l2vpn/1.0/c73da8e1-5d03-4620-a1db-7cdf23e8978c
SDXException: SDXException: L2VPN Service ID provided does not exist (status_code=404)


**And here I expect to see a response that I don't have authorization to delete this l2vpn.**

In [23]:
with requests_mock.Mocker() as mock_requests:
    # Mock response for 401: not authorized
    mock_requests.delete(
        f"{url}/l2vpn/{client.VERSION}/{service_id}",
        status_code=401,
        json={"description": "Not Authorized"}
    )
    
    print(f"Request URL: {url}/l2vpn/{client.VERSION}/{service_id}")

    try:
        response = client.delete_l2vpn(service_id)
        print("Delete L2VPN (Not Authorized):")
        pprint(response)
    except SDXException as e:
        print("SDXException:", e)

Failed to delete L2VPN. Status code: 401: Not Authorized


Request URL: http://aw-sdx-controller.renci.org:8081/l2vpn/1.0/c73da8e1-5d03-4620-a1db-7cdf23e8978c
SDXException: SDXException: Not Authorized (status_code=401)


# Topology

In [24]:
# This portion loops over the list of l2vpns that are in use and 
# creates sorted lists for each one with their correponding 
# vlans that are being used. 

# Start the mock request session
with requests_mock.Mocker() as mock_requests:
    # Mock the GET request for retrieving active L2VPNs
    mock_requests.get(f"{url}/l2vpn/1.0", json=mock_response_active_l2vpns_exist)

    # Initialize SDXClient with the mock URL
    client = SDXClient(base_url=url)

    # Call `_get_vlans_in_use()` to retrieve VLAN usage
    vlan_usage = client._get_vlans_in_use()

    # Print and verify the output
    print("\n**VLANs in Use Per Port:**")
    for port, vlans in vlan_usage.items():
        print(f"Port: {port} → VLANs: {vlans}")


**VLANs in Use Per Port:**
Port: urn:sdx:port:tenet.ac.za:Tenet03:50 → VLANs: [150, 3500]
Port: urn:sdx:port:sax.net:Sax02:50 → VLANs: [400]
Port: urn:sdx:port:ampath.net:Ampath3:50 → VLANs: [300, 3500]
Port: urn:sdx:port:sax.net:Sax01:50 → VLANs: [3500]


In [25]:
# This will demonstrate the return of the vlans that are available.

# Convert the topology dictionary into an SDXTopologyResponse object
topology_response = SDXTopologyResponse.from_json(topology)

# Extract ports from the topology for testing
ports = {
    port["id"]: Port(
        id=port["id"],
        name=port["name"],
        node=port["node"],
        status=port["status"],
        state=port["state"],
        nni=port.get("nni", ""),
        type=port["type"],
        services=port["services"]
    )
    for node in topology["nodes"] for port in node["ports"]
}

# Mock API responses
with requests_mock.Mocker() as mock:
    # Mock /topology API response
    mock.get(f"{url}/topology", json=topology)

    # Mock /l2vpn/1.0 GET (Fetching active L2VPNs)
    mock.get(f"{url}/l2vpn/1.0", json=mock_response_active_l2vpns_exist)

    # Mock get_topology() method to return topology_response
    client.get_topology = lambda: topology_response

    # Mock get_all_l2vpns() method to return VLANs in use
    client.get_all_l2vpns = lambda format="json": mock_response_active_l2vpns_exist

    # Compute VLANs in use
    vlans_in_use = client._get_vlans_in_use()

    # Test _get_vlan_range for a specific port
    test_port_id = "urn:sdx:port:ampath.net:Ampath3:50"
    test_port = ports[test_port_id]

    # Compute available VLANs
    available_vlans = client._get_vlan_range(test_port, vlans_in_use)

    # Print Output
    print("\n**VLANs in Use Per Port:**")
    for port_id, vlans in vlans_in_use.items():
        print(f"Port: {port_id} → VLANs: {vlans}")

    print(f"\n**Available VLANs for Port {test_port_id}**")
    print(available_vlans)



**VLANs in Use Per Port:**
Port: urn:sdx:port:tenet.ac.za:Tenet03:50 → VLANs: [150, 3500]
Port: urn:sdx:port:sax.net:Sax02:50 → VLANs: [400]
Port: urn:sdx:port:ampath.net:Ampath3:50 → VLANs: [300, 3500]
Port: urn:sdx:port:sax.net:Sax01:50 → VLANs: [3500]

**Available VLANs for Port urn:sdx:port:ampath.net:Ampath3:50**
1-299, 301-3499, 3501-4095


In [26]:
# Mock API responses
with requests_mock.Mocker() as mock:
    # Mock /topology API response
    mock.get(f"{url}/topology", json=topology)

    # Mock /l2vpn/1.0 GET (Fetching active L2VPNs)
    mock.get(f"{url}/l2vpn/1.0", json=mock_response_l2vpn_exists)

    ### Display Available Ports
    available_ports_df = client.get_available_ports()

    ### Print Outputs
    print("\n**Available Ports and VLANs**")
    display(available_ports_df)



**Available Ports and VLANs**


,Port ID,Domain,Device,Port #,Status,Entities,VLANs Available,VLANs in Use
0,urn:sdx:port:ampath.net:Ampath3:50,ampath.net,Ampath3,50,up,"Disney World, University of Florida, MIT","1-299, 301-3499, 3501-4095",300; 3500
1,urn:sdx:port:ampath.net:Ampath2:50,ampath.net,Ampath2,50,up,"NCSA, UIUC, University of Illinois",1-4095,None
2,urn:sdx:port:ampath.net:Ampath1:50,ampath.net,Ampath1,50,up,"Canada, New York, University",1-4095,None
3,urn:sdx:port:sax.net:Sax01:50,sax.net,Sax01,50,up,"France, Cern","1-3499, 3501-4095",3500
4,urn:sdx:port:sax.net:Sax02:50,sax.net,Sax02,50,up,"UCLA, University of California, Los Angeles, Los Angeles","1-399, 401-4095",400
5,urn:sdx:port:tenet.ac.za:Tenet02:50,tenet.ac.za,Tenet02,50,up,"FSU, Florida State University, Tallahassee",1-4095,None
6,urn:sdx:port:tenet.ac.za:Tenet03:50,tenet.ac.za,Tenet03,50,up,"NYU, New York University, New York City","1-149, 151-3499, 3501-4095",150; 3500
7,urn:sdx:port:tenet.ac.za:Tenet01:50,tenet.ac.za,Tenet01,50,up,"GT, Georgia Tech, Atlanta",1-4095,None


# Search Ports

In [27]:
# Create the client instance
client = SDXClient(base_url=url)

# Mock API responses
with requests_mock.Mocker() as mock:
    mock.get(f"{url}/topology", json=topology)  # Mock topology response

    # Run the search for "MIT"
    search_term = "Ampath"
    search_results_df = client.search_ports(search_term)

    # Display the search results
    print("\n**Search Results for Ports Matching:**", search_term)
    display(search_results_df)  # Show the DataFrame nicely in a Jupyter Notebook


**Search Results for Ports Matching:** Ampath


,name,id,node,type,status,state,mtu,services,entities
0,Ampath3-eth50,urn:sdx:port:ampath.net:Ampath3:50,urn:sdx:node:ampath.net:Ampath3,PortType.GE_10,Status.UP,State.ENABLED,1500,"{'l2vpn-ptp': {'vlan_range': [[1, 4095]]}, 'l2vpn-ptmp': {}}","[Disney World, University of Florida, MIT]"
1,Ampath2-eth50,urn:sdx:port:ampath.net:Ampath2:50,urn:sdx:node:ampath.net:Ampath2,PortType.GE_10,Status.UP,State.ENABLED,1500,"{'l2vpn-ptp': {'vlan_range': [[1, 4095]]}, 'l2vpn-ptmp': {}}","[NCSA, UIUC, University of Illinois]"
2,Ampath1-eth50,urn:sdx:port:ampath.net:Ampath1:50,urn:sdx:node:ampath.net:Ampath1,PortType.GE_10,Status.UP,State.ENABLED,1500,"{'l2vpn-ptp': {'vlan_range': [[1, 4095]]}, 'l2vpn-ptmp': {}}","[Canada, New York, University]"


# Search Entities

In [28]:
# Create the client instance
client = SDXClient(base_url=url)

# Mock API responses
with requests_mock.Mocker() as mock:
    mock.get(f"{url}/topology", json=topology)  # Mock topology response

    # Run the search for "MIT"
    search_term = "MIT"
    search_results_df = client.search_entities(search_term)

    # Display the search results
    print("\n**Search Results for Entities Matching:**", search_term)
    display(search_results_df)  # Show the DataFrame nicely in a Jupyter Notebook



**Search Results for Entities Matching:** MIT


,name,id,node,type,status,state,mtu,services,entities
0,Ampath3-eth50,urn:sdx:port:ampath.net:Ampath3:50,urn:sdx:node:ampath.net:Ampath3,PortType.GE_10,Status.UP,State.ENABLED,1500,"{'l2vpn-ptp': {'vlan_range': [[1, 4095]]}, 'l2vpn-ptmp': {}}","[Disney World, University of Florida, MIT]"


# General Search Ports or Entities

In [35]:
# Create the client instance
client = SDXClient(base_url=url)

# Mock API responses
with requests_mock.Mocker() as mock:
    mock.get(f"{url}/topology", json=topology)  # Mock topology response

    mock.get(f"{url}/l2vpn/1.0", json=mock_response_active_l2vpns_exist)
    
    # client.search("Ampath", "ports")     # Calls search_ports()
    result = client.search("Uni", "entities")     # Calls search_entities()

result

,Port ID,Domain,Device,Port #,Status,Entities,VLANs Available,VLANs in Use
0,urn:sdx:port:ampath.net:Ampath3:50,ampath.net,Ampath3,50,up,"Disney World, University of Florida, MIT","1-299, 301-3499, 3501-4095",300; 3500
1,urn:sdx:port:ampath.net:Ampath3:3,ampath.net,Ampath3,3,up,"Indiana University, IU",1-4095,None
2,urn:sdx:port:ampath.net:Ampath2:50,ampath.net,Ampath2,50,up,"NCSA, UIUC, University of Illinois",1-4095,None
3,urn:sdx:port:ampath.net:Ampath1:1,ampath.net,Ampath1,1,up,"South America, Universidad De Brazil",1-4095,None
4,urn:sdx:port:ampath.net:Ampath1:50,ampath.net,Ampath1,50,up,"Canada, New York, University",1-4095,None
5,urn:sdx:port:sax.net:Sax02:41,sax.net,Sax02,41,up,"TTU, Tennessee Tech Univ, TN Tech University, Cookeville",1-4095,None
6,urn:sdx:port:sax.net:Sax02:1,sax.net,Sax02,1,up,"FIU, Florida International Unitversity, Miami",1-4095,None
7,urn:sdx:port:sax.net:Sax02:50,sax.net,Sax02,50,up,"UCLA, University of California, Los Angeles, Los Angeles","1-399, 401-4095",400
8,urn:sdx:port:tenet.ac.za:Tenet02:41,tenet.ac.za,Tenet02,41,up,"UT, University of Texas, Austin",1-4095,None
9,urn:sdx:port:tenet.ac.za:Tenet02:1,tenet.ac.za,Tenet02,1,up,"OSU, Ohio State University, Columbus",1-4095,None


In [30]:
# Create the client instance
client = SDXClient(base_url=url)

# Mock API responses
with requests_mock.Mocker() as mock:
    mock.get(f"{url}/topology", json=topology)  # Mock topology response

    mock.get(f"{url}/l2vpn/1.0", json=mock_response_active_l2vpns_exist)
    
    result = client.search("Sa", "ports")     # Calls search_ports()
    # client.search("FIU", "entities")     # Calls search_entities()

result

,Port ID,Domain,Device,Port #,Status,Entities,VLANs Available,VLANs in Use
0,urn:sdx:port:sax.net:Sax01:41,sax.net,Sax01,41,up,None,1-4095,None
1,urn:sdx:port:sax.net:Sax01:1,sax.net,Sax01,1,up,Sao Paulo,1-4095,None
2,urn:sdx:port:sax.net:Sax01:40,sax.net,Sax01,40,up,"Sao Paulo, Ampath",1-4095,None
3,urn:sdx:port:sax.net:Sax01:50,sax.net,Sax01,50,up,"France, Cern","1-3499, 3501-4095",3500
4,urn:sdx:port:sax.net:Sax02:41,sax.net,Sax02,41,up,"TTU, Tennessee Tech Univ, TN Tech University, Cookeville",1-4095,None
5,urn:sdx:port:sax.net:Sax02:1,sax.net,Sax02,1,up,"FIU, Florida International Unitversity, Miami",1-4095,None
6,urn:sdx:port:sax.net:Sax02:40,sax.net,Sax02,40,up,"MIT, Massachusetts Institute of Technology, Cambridge",1-4095,None
7,urn:sdx:port:sax.net:Sax02:50,sax.net,Sax02,50,up,"UCLA, University of California, Los Angeles, Los Angeles","1-399, 401-4095",400


# Interactive Search

In [31]:
    
client.interactive_search()

Interactive Mode Enabled: Select search type and enter a query.


Dropdown(description='Type:', options=('ports', 'entities'), value='ports')

Text(value='', continuous_update=False, description='Search:', placeholder='Enter a search term...')